In [1]:
import argparse
import yaml
import os, sys
import time
import logging, logging.config

import pandas as pd
import geopandas as gpd
import rasterio

import stat
import math

from tqdm import tqdm

from misc_fct import test_crs

/home/gsalamin/Python_env/road_surfaces/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [3]:
with open('config.yaml') as fp:
    cfg = yaml.load(fp, Loader=yaml.FullLoader)['statistical_analysis.py']    #  [os.path.basename(__file__)]


# Defitions of the functions


# Definition of the constants
INPUT_FOLDER=cfg['input']['input_folder']
PROCESSED=cfg['processed']
PROCESSED_FOLDER=PROCESSED['processed_folder']
FINAL=cfg['final']
FINAL_FOLDER=FINAL['final_folder']

# Inputs
ROADS=PROCESSED_FOLDER + PROCESSED['input_files']['roads']
TILES_DIR=PROCESSED_FOLDER + PROCESSED['input_files']['images']
TILES_INFO=PROCESSED_FOLDER + PROCESSED['input_files']['tiles']

# Outputs
STATS_ROADS=PROCESSED_FOLDER + PROCESSED['output_files']['stats']
STATS_TYPE=FINAL_FOLDER+ FINAL['stats_by_type']

# Main

## Import data

In [4]:
# Importation of the files
roads=gpd.read_file(ROADS)
tiles_info = gpd.read_file(TILES_INFO)

In [18]:
print(roads.columns)
print(tiles_info.columns)
roads.shape[0]


Index(['OBJECTID', 'DATUM_AEND', 'DATUM_ERST', 'ERSTELLUNG', 'ERSTELLU_1',
       'REVISION_J', 'REVISION_M', 'GRUND_AEND', 'HERKUNFT', 'HERKUNFT_J',
       'HERKUNFT_M', 'REVISION_Q', 'OBJEKTART', 'KUNSTBAUTE', 'WANDERWEGE',
       'VERKEHRSBE', 'BEFAHRBARK', 'EROEFFNUNG', 'STUFE', 'RICHTUNGSG',
       'BELAGSART', 'KREISEL', 'EIGENTUEME', 'VERKEHRS_1', 'NAME',
       'TLM_STRASS', 'STRASSENNA', 'SHAPE_Leng', 'Width', 'geometry'],
      dtype='object')
Index(['title', 'id', 'geometry'], dtype='object')


11179

## Data treatment

In [42]:
print(roads[roads.is_valid==False].shape[0])          # to_file(PROCESSED_FOLDER + '/shapefiles_gpkg/test_invalid_geom.shp')

valid_roads=roads[roads.is_valid==True].drop(columns=['ERSTELLUNG', 'ERSTELLU_1', 'HERKUNFT', 'HERKUNFT_J', 'HERKUNFT_M','KUNSTBAUTE', 'WANDERWEGE',
       'VERKEHRSBE', 'BEFAHRBARK', 'EROEFFNUNG', 'STUFE', 'RICHTUNGSG', 'KREISEL', 'EIGENTUEME', 'VERKEHRS_1', 'NAME', 'TLM_STRASS', 'STRASSENNA', 
       'SHAPE_Leng', 'Width'])

## Trouver pourquoi certaines géométries ne sont plus valides après l'importation et corriger ça!



In [43]:
valid_roads.to_crs(epsg=3857, inplace=True)
tiles_info.to_crs(epsg=3857, inplace=True)
test_crs(valid_roads.crs, tiles_info.crs)

print(valid_roads[valid_roads.is_valid==False].shape[0])

clipped_roads = gpd.clip(valid_roads, tiles_info).explode()

# clipped_roads=gpd.GeoDataFrame()
# for idx in tqdm(tiles_info.index, desc='Clipping roads'):

#     roads_to_tile = gpd.clip(roads_poly, tiles_info.loc[idx,'geometry']).explode()

#     clipped_roads=pd.concat([clipped_roads,roads_to_tile], ignore_index=True)


0


/tmp/ipykernel_153184/1676487618.py:7: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  clipped_roads = gpd.clip(valid_roads, tiles_info).explode()


### Calcul et analyse des statistiques de zone pour les routes

#### PCA

### Calcul et analyses des statistiques par types

#### Boxplots